<a href="https://colab.research.google.com/github/VedantDgit/Crop-and-fertilizer-Recommend-model-for-Amravati-MH-/blob/main/Crop_and_fertilizer_Recommend_model_for_Amravati_Areaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("/content/FinalAmravati Crop and fertilizer dataset.csv")

In [ ]:
dataset

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.duplicated().sum()

In [ ]:
dataset.describe()

In [ ]:
numeric_dataset = dataset.select_dtypes(include=['number'])
corr = numeric_dataset.corr()
display(corr)

In [ ]:
import seaborn as sns
sns.heatmap(corr, annot=True, cmap='coolwarm')

Importing libraries that required for main implementation

In [ ]:
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
taluka_widget = widgets.Combobox(
    placeholder='Choose Taluka',
    options=tuple(dataset['Taluka_Name'].unique()),
    description='Select Taluka:',
    ensure_option=True,
    disabled=False
)

soil_color_widget = widgets.Combobox(
    placeholder='Select Soil Color of region',
    options=tuple(dataset['Soil_color'].unique()),
    description='Soil Color:',
    ensure_option=True
)

nitrogen_widget = widgets.Combobox(
    description= "Nitrogen:",
    placeholder='Select Nitrogen',
    ensure_option=True,
)

phosphorus_widget = widgets.Combobox(
    description= "Phosphorus:",
    placeholder='Select Phosphorus',
    ensure_option=True,
)

potassium_widget = widgets.Combobox(
    description= "Potassium:",
    placeholder='Select Potassium',
    ensure_option=True,
)

ph_widget = widgets.Combobox(
    description= "pH:",
    placeholder='Select pH',
    ensure_option=True,
)

rainfall_widget = widgets.Combobox(
    description= "Rainfall:",
    placeholder='Select Rainfall',
    ensure_option=True,
)

temperature_widget = widgets.Combobox(
    description= "Temperature:",
    placeholder='Select Temperature',
    ensure_option=True,
)

recommend_widget = widgets.Output()

# Define the observer function to update soil color options
def update_soil_color_options(change):
    taluka = change.new
    if taluka:
        soil_colors = dataset[dataset['Taluka_Name'] == taluka]['Soil_color'].unique()
        soil_color_widget.options = tuple(soil_colors)
    else:
        soil_color_widget.options = ()

# Define the observer function to update nitrogen options
def update_nitrogen_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        nitrogen_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['Nitrogen'].unique()
        nitrogen_widget.options = tuple(str(value) for value in nitrogen_values)  # Convert to Unicode strings
    else:
        nitrogen_widget.options = ()

# Define the observer function to update phosphorus options
def update_phosphorus_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        phosphorus_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['Phosphorus'].unique()
        phosphorus_widget.options = tuple(str(value) for value in phosphorus_values)  # Convert to Unicode strings
    else:
        phosphorus_widget.options = ()

def update_potassium_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        potassium_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['Potassium'].unique()
        potassium_widget.options = tuple(str(value) for value in potassium_values)  # Convert to Unicode strings
    else:
        potassium_widget.options = ()

def update_ph_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        ph_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['pH'].unique()
        ph_widget.options = tuple(str(value) for value in ph_values) # Convert to Unicode strings
    else:
        ph_widget.options = ()

def update_rainfall_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        rainfall_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['Rainfall'].unique()
        rainfall_widget.options = tuple(str(value) for value in rainfall_values) # Convert to Unicode strings
    else:
        rainfall_widget.options = ()

def update_temperature_options(change):
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    if taluka and soil_color:
        temperature_values = dataset[(dataset['Taluka_Name'] == taluka) & (dataset['Soil_color'] == soil_color)]['Temperature'].unique()
        temperature_widget.options = tuple(str(value) for value in temperature_values) # Convert to Unicode strings
    else:
        temperature_widget.options = ()

# Train the model
def train_model(change):
    # Get the selected values from the combo boxes
    taluka = taluka_widget.value
    soil_color = soil_color_widget.value
    nitrogen = float(nitrogen_widget.value)
    phosphorus = float(phosphorus_widget.value)
    potassium = float(potassium_widget.value)
    pH = float(ph_widget.value)
    rainfall = float(rainfall_widget.value)
    temperature = float(temperature_widget.value)

    input_data = pd.DataFrame(
        [[nitrogen, phosphorus, potassium, pH, rainfall, temperature, taluka, soil_color]],
        columns=['Nitrogen', 'Phosphorus', 'Potassium', 'pH', 'Rainfall', 'Temperature', 'Taluka_Name', 'Soil_color']
    )

    # Perform one-hot encoding for District_Name and Soil_color columns
    encoder = OneHotEncoder(handle_unknown='ignore')
    X_encoded = encoder.fit_transform(dataset[['Taluka_Name', 'Soil_color']])
    input_data_encoded = encoder.transform(input_data[['Taluka_Name', 'Soil_color']])

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, dataset['Crop'], test_size=0.2, random_state=42)

    # Train the Random Forest model
    model_crop = RandomForestClassifier(n_estimators=100, random_state=42)
    model_crop.fit(X_train, y_train)

    #Make predictions
    predicted_crop = model_crop.predict(input_data_encoded)

    # Find the fertilizer associated with the recommended crop
    recommended_fertilizer = dataset[dataset['Crop'] == predicted_crop[0]]['Fertilizer'].values[0]
    with recommend_widget:
        recommend_widget.clear_output()
        print("Recommended Crop:", predicted_crop[0])
        print("Recommended Fertilizer:", recommended_fertilizer)


# Define the observer function to update soil color options
taluka_widget.observe(update_soil_color_options, names='value')
taluka_widget.observe(update_nitrogen_options, names='value')
soil_color_widget.observe(update_nitrogen_options, names='value')

taluka_widget.observe(update_phosphorus_options, names='value')
soil_color_widget.observe(update_phosphorus_options, names='value')

taluka_widget.observe(update_potassium_options, names='value')
soil_color_widget.observe(update_potassium_options, names='value')

taluka_widget.observe(update_ph_options, names='value')
soil_color_widget.observe(update_ph_options, names='value')

taluka_widget.observe(update_rainfall_options, names='value')
soil_color_widget.observe(update_rainfall_options, names='value')

taluka_widget.observe(update_temperature_options, names='value')
soil_color_widget.observe(update_temperature_options, names='value')

#create button widget
button = widgets.Button(description="Recommend Crop and Fertilizer")
button.on_click(train_model)

display(widgets.VBox([
    taluka_widget,
    soil_color_widget,
    nitrogen_widget,
    phosphorus_widget,
    potassium_widget,
    ph_widget,
    rainfall_widget,
    temperature_widget]))

display(button)

In [ ]:
display(recommend_widget)